Assumptions made:
1. Since flight time is not an attribute, we assume that each flight can only fly once in a day.
2. It is assumed that *atLeastNTimes: Int* in the extra question refer to more than N and does not include N.




In [1]:
import pandas as pd

In [2]:
flights = pd.read_csv('flightData.csv')
flights.head()

,passenger_id,flight_id,from,to,date
0,1,1,SG,MY,01/01/2025
1,2,1,SG,MY,01/01/2025
2,2,2,MY,CN,02/01/2025
3,3,2,MY,CN,02/02/2234
4,4,2,MY,CN,02/01/2025


In [3]:
passengers = pd.read_csv('passengers.csv')
passengers.head()

,passenger_id,first_name,last_name
0,1,Adam,Smith
1,2,Xavier,Bennett
2,3,Olivia,Carter
3,4,Jia Jia,Lee
4,5,Brianna,Howard


In [4]:
print(flights.columns)
print(passengers.columns)

Index(['passenger_id', 'flight_id', 'from ', 'to', 'date'], dtype='object')
Index(['passenger_id', 'first_name', 'last_name'], dtype='object')


In [5]:
flights['date'] = pd.to_datetime(flights['date'], format='%d/%m/%Y')
flights['month'] = flights['date'].dt.month
flights['month'].value_counts().sort_index().rename(index={1: "January", 2: "February"})

,count
month,
January,189
February,146


In [6]:
passenger_counts = flights['passenger_id'].value_counts().reset_index()
passenger_counts.columns = ['passenger_id', 'Number of Flights']

passenger_counts = passenger_counts.sort_values(by=['Number of Flights'], ascending=[False])

#Find the number of flight of the 10th person
threshold = passenger_counts.iloc[9]['Number of Flights']

#Anyone who is more than or equal to the value of the 10th person will also be added to the table
top_passengers = passenger_counts[passenger_counts['Number of Flights'] >= threshold]


result = pd.merge(top_passengers, passengers, on='passenger_id', how='left')
result = result[['passenger_id', 'Number of Flights', 'first_name', 'last_name']]

print(result)

# Since the last 4 passengers took the same number of flights, all the 4 passengers will be included.

    passenger_id  Number of Flights first_name last_name
0             72                  8    Chelsea      Kwek
1             92                  8    Cameron     Evans
2            129                  8       Enna     Chong
3             38                  6    Tristan       Cox
4             14                  6    Vanessa       Lim
5             10                  6      Caleb   Sanders
6             35                  5   Kai Xuan       Ong
7              6                  5     Damien       Ong
8              5                  4    Brianna    Howard
9            172                  4    Sarbina       Loh
10             8                  4      Ethan       Koh
11           176                  4     Emmett     Scott


In [7]:
# Only these 3 attributes are important for this question
longestrun = flights[['passenger_id', 'from ', 'to']]

# Function to compute the longest sequence of unique countries without SG
def longest_run_without_sg(countries):
  max_run = 0
  current_run = set()

  for country in countries:
    # Reset when we hit SG
    if country == 'SG':
      current_run.clear()
    else:
      # Add to current run if it is not 'SG'
      current_run.add(country)
      max_run = max(max_run, len(current_run))
  return max_run


# Function to deal with each passenger
def get_longest_run(group):
  # List all the countries from 'from' column and the last country from 'to' column
  travel_sequence = list(group['from ']) + [group['to'].iloc[-1]]
  return longest_run_without_sg(travel_sequence)


# Group by 'passenger_id' and apply get_longest_run function
longest_runs = longestrun.groupby('passenger_id', group_keys=False).apply(get_longest_run, include_groups=False).reset_index()
longest_runs.columns = ['passenger_id', 'Longest Run']


pd.set_option('display.max_rows', None)
print(longest_runs)


     passenger_id  Longest Run
0               1            1
1               2            2
2               3            2
3               4            2
4               5            2
5               6            3
6               7            1
7               8            3
8               9            2
9              10            1
10             11            2
11             12            1
12             13            2
13             14            5
14             15            1
15             16            2
16             17            2
17             18            1
18             19            2
19             20            1
20             21            2
21             22            1
22             23            1
23             24            1
24             25            1
25             26            1
26             27            1
27             28            1
28             29            2
29             30            1
30             31            2
31      

In [8]:
from itertools import combinations

freq_pairs = flights[['passenger_id', 'flight_id', 'date']]
pairs = []

# Group by 'flight_id' and 'date' to find passengers who took the same flight
for _, group in freq_pairs.groupby(['flight_id', 'date']):
  # Sort to avoid duplicate ordering
  passenger_ids = sorted(group['passenger_id'].tolist())
  for pair in combinations(passenger_ids, 2):
    pairs.append(pair)


pairs_df = pd.DataFrame(pairs, columns=['Passenger 1', 'Passenger 2'])

# Count occurrences of each pair
pair_counts = pairs_df.value_counts().reset_index()
pair_counts.columns = ['Passenger 1', 'Passenger 2', 'Flight Count']

# Filter pairs who have been on more than 2 flights together
frequent_pairs = pair_counts[pair_counts['Flight Count'] > 2]


print(frequent_pairs)



   Passenger 1  Passenger 2  Flight Count
0           10          129             6
1           14           92             6
2            6           38             4
3          172          176             3


In [ ]:
def flownTogether(flightData: pd.DataFrame, atLeastNTimes: int, startDate: str, endDate: str):


  flightData['date'] = pd.to_datetime(flightData['date'], format='%d/%m/%Y')

  # Filter flights within the specified date range
  filtered_flights = flightData[(flightData['date'] >= pd.to_datetime(startDate, format='%d/%m/%Y')) &
                                  (flightData['date'] <= pd.to_datetime(endDate, format='%d/%m/%Y'))]

  pairs = []

  # Group by 'flight_id' and 'date' to find passengers who took the same flight (Similar to the previous question)
  for _, group in filtered_flights.groupby(['flight_id', 'date']):
        # Sort to avoid duplicate ordering
        passenger_ids = sorted(group['passenger_id'].tolist())
        for pair in combinations(passenger_ids, 2):
            pairs.append(pair)


  pairs_df = pd.DataFrame(pairs, columns=['Passenger 1', 'Passenger 2'])

  pair_counts = pairs_df.value_counts().reset_index()
  pair_counts.columns = ['Passenger 1', 'Passenger 2', 'Flight Count']
  frequent_pairs = pair_counts[pair_counts['Flight Count'] > atLeastNTimes]

  return frequent_pairs


flightData = pd.read_csv('flightData.csv')

# Get user inputs
print("To find the passengers who have been on more than N flights together,")
atLeastNTimes = int(input("Enter the value of N: "))
startDate = input("Enter start date (dd/mm/yyyy): ")
endDate = input("Enter end date (dd/mm/yyyy): ")

result = flownTogether(flightData, atLeastNTimes, startDate, endDate)

print(result)

To find the passengers who have been on more than N flights together,
